In [28]:
"""
preparation.ipynb
作成対象のバス会社の情報を入力し、GTFS生成材料を作成する
▽作成ファイル▽
material.json:バス会社の基本情報
material.pickle:latlon付きnavitimeバス停情報のテーブル
"""

import pandas as pd
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import os
import pickle
import uuid
import time

from pathlib import Path

import geopandas as gpd

from fuzzywuzzy import process

"""
FUNCTION
"""
def adj(law): # navitimeのユニーク化の処理を除外する
    if "(" in law:
        return law.split("(")[0]
    elif "〔" in law:
        return law.split("〔")[0]
    else:
        return law
    
def dict_to_json(dir:str,data:dict):
    """
    @dir    :書き込み先ファイルパス
    @data   :任意の辞書型データ
    """ 
    import json
    with open(dir, mode="wt", encoding="utf-8") as f:
 	    json.dump(data, f, ensure_ascii=False, indent=2)

In [5]:
"""
INPUT
基本情報の手入力
"""
# 対象バス会社名（国土数値情報における定義）
NAME_OF_BUS_COMPANY = "東運輸（株）"
# バス会社のnavitimeトップページ
URL_BUS_COMPANY_PAGE = "https://www.navitime.co.jp/bus/company/00001313/"
# 対象バス会社が属する都道府県の国土数値情報バス停情報.shp
DIR_KOKUDO_SUCHI_BUS_STOP_INFO = "C:\\lab\\gtfses\\Ishigaki_pj\\busStop_KokudoSuuchi\\P11-22_47_SHP\\P11-22_47.shp"
# 作業ディレクトリの定義
DIR_WRITING = "C:/lab/gtfses/Ishigaki_pj"

# バス会社名確認用
# tmp = gpd.read_file(DIR_KOKUDO_SUCHI_BUS_STOP_INFO)
# tmp["P11_002"].unique()

In [6]:
"""
当該バス会社の全バス停情報をnavitimeから取得するプログラム
"""
# 結果格納用辞書 key:nameOfBusStop value:link
bsInfoAndLinks = []
# 平仮名とリンク
kana_link_dict = {}


# バス停の平仮名からスクレイピング
response = requests.get(URL_BUS_COMPANY_PAGE)
soup = BeautifulSoup(response.text, "html.parser")
links = soup.find_all("dl", class_="kana-frame")

for link in links:
    soup_c = BeautifulSoup(str(link), "html.parser")

    for a in soup_c.select("a"):
        kana_link_dict[a.text] = a["href"]

# バス停名とリンクを取得
for kana_link in list(kana_link_dict.values()):        
    response = requests.get("https://" + kana_link[2:])
    soup = BeautifulSoup(response.text,"html.parser")
    node_contents = soup.find("ul",class_="node-list")
    nodes = BeautifulSoup(str(node_contents),"html.parser").find_all("li")
    for node in nodes:
        soup_c = BeautifulSoup(str(node),"html.parser")
        url = soup_c.find("a")["href"]
        nameBusStop = soup_c.find("rb").text
        id = url.split("/")[-2]
        #結果格納
        bsInfoAndLinks.append({"nameBS_navitime":nameBusStop,"url":url,"id":id})

In [12]:
"""
navitimeから取得した情報にlatlonを付与（国土数値情報との突合）
"""
# 国土数値情報からバス会社名でクエリして、必要なカラムを抽出
queried = gpd.read_file(DIR_KOKUDO_SUCHI_BUS_STOP_INFO).filter(items=["P11_001","P11_002","geometry"]).set_axis(["nameBS_kokudo","nameOfCompany","geometry"],axis=1).query("nameOfCompany == @NAME_OF_BUS_COMPANY")

# merge1 navitimeのユニーク化処理を回避してマージ
navitimeDf = pd.DataFrame(bsInfoAndLinks) # スクレイピングした情報をpandas.DataFrameに
navitimeDf["nameBS_navitime_adj"] = navitimeDf["nameBS_navitime"].map(adj) # navitimeのユニーク化の処理を除外する
merged = pd.merge(navitimeDf,queried,left_on="nameBS_navitime_adj",right_on="nameBS_kokudo",how="left")
matched = merged.query("nameBS_kokudo.notnull()")
unMatched = merged.query("nameBS_kokudo.isnull()").filter(items=["nameBS_navitime","url","id","nameBS_navitime_adj"])

# merge2 言語処理で近いものをマージ
choices = list(set(queried["nameBS_kokudo"].to_list()) - set(matched["nameBS_kokudo"].to_list()))
unMatched["expected"] = unMatched["nameBS_navitime_adj"].apply(lambda x:process.extractOne(x,choices)[0])
unMatched

,nameBS_navitime,url,id,nameBS_navitime_adj,expected
2,新川公園,//www.navitime.co.jp/bus/diagram/direction/004...,00428163,新川公園,新川公園前
5,新川団地(沖縄県),//www.navitime.co.jp/bus/diagram/direction/004...,00428215,新川団地,新川団地前
17,伊野田３班,//www.navitime.co.jp/bus/diagram/direction/004...,00428132,伊野田３班,伊野田二班
18,伊野田２班,//www.navitime.co.jp/bus/diagram/direction/004...,00428131,伊野田２班,伊野田二班
22,ＡＮＡインターコンチネンタル(石垣市),//www.navitime.co.jp/bus/diagram/direction/004...,00428188,ＡＮＡインターコンチネンタル,ANAインターコンチネンタル
30,大浜信泉記念館,//www.navitime.co.jp/bus/diagram/direction/004...,00428100,大浜信泉記念館,大浜信泉記念館前
35,川平公園,//www.navitime.co.jp/bus/diagram/direction/004...,00428193,川平公園,川平公園前
36,川平郵便前,//www.navitime.co.jp/bus/diagram/direction/004...,00428183,川平郵便前,川平郵便局前
42,グランヴィリオ,//www.navitime.co.jp/bus/diagram/direction/004...,00428211,グランヴィリオ,グランヴィリオリゾート石垣島
50,サッカーパーク前,//www.navitime.co.jp/bus/diagram/direction/004...,00428152,サッカーパーク前,サッカーパーク赤馬


In [16]:
"""
上記で突合できなかったものを手作業で突合
"""
# pd.set_option("display.max_row",150)
unMatched.at[129,"expected"] = "米原キャンプ場"	

In [27]:
"""
unMatchとmatchの結合
"""
unMatched_adj   = pd.merge(unMatched,queried,left_on="expected",right_on="nameBS_kokudo",how="left")
geoNavitime     = pd.concat([matched,unMatched_adj],axis=0).filter(items=["nameBS_navitime","url","id","geometry"])
gdfNavitime     = gpd.GeoDataFrame(geoNavitime,geometry="geometry")
gdfNavitime["lon"], gdfNavitime["lat"] = gdfNavitime["geometry"].x, gdfNavitime["geometry"].y
materialPickle  = gdfNavitime.drop(columns=("geometry"))

In [29]:
"""
OUTPUT
material.json
material.pickle
の出力
"""
materialJson = {
    "NAME_OF_BUS_COMPANY": NAME_OF_BUS_COMPANY,
    "URL_BUS_COMPANY_PAGE": URL_BUS_COMPANY_PAGE,
    "DIR_WRITING": DIR_WRITING
}
dict_to_json(DIR_WRITING+"/material.json",materialJson) 
materialPickle.to_pickle(DIR_WRITING+"/material.pickle")